In [25]:
#part one: compute the jaccard similarity
#https://github.com/VipanchiKatthula/Jaccard_Cosine_Similarity/blob/master/Jaccard_and_Cosine_Similarity.ipynb
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 KB 6.1 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

In [2]:
#load the file
df_hsien = pd.read_csv('/Users/wangxuechun/unsw/UNSW-Thesis-/chapter_4_eng_version_one.csv')
df_google = pd.read_csv('/Users/wangxuechun/unsw/UNSW-Thesis-/chapter_4_trans.csv')
df_google = df_google.drop('Chinese_verse', axis=1)
df_yiyun = pd.read_csv('/Users/wangxuechun/unsw/UNSW-Thesis-/chapter_4_eng_version_yiyun.csv')
df_hsien.columns = ['English_Verse']
df_yiyun.columns = ['English_Verse']

In [3]:
#convert lists of lists:
def lists_of_lists(lists):
    return [item for sublist in lists for item in sublist]

In [4]:
#computation of jaccard similarities
def jaccard_similarities(list1, list2):
    intersections = len(set(list1).intersection(set(list2)))
    unions = len(set(list1).union(set(list2)))
    return (intersections/unions)*100

In [5]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt')
nltk.download('stopwords')





[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wangxuechun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wangxuechun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#data cleaning
def data_cleaning(text):
    tokens = word_tokenize(text)
    #remove the punctuation and convert all to the lower case
    tokens = [token.lower() for token in tokens if token.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_text = [word for word in tokens if word not in stop_words]
    return filtered_text


hsien_lists = df_hsien['English_Verse'].apply(data_cleaning)
google_lists = df_google['English_Verse'].apply(data_cleaning)
yiyun_lists = df_yiyun['English_Verse'].apply(data_cleaning)

hsien_list = lists_of_lists(hsien_lists)
google_list = lists_of_lists(google_lists)
yiyun_list = lists_of_lists(yiyun_lists)

['said', 'victors', 'take', 'pleasure', 'victory', 'unless', 'opponents', 'arc', 'fierce', 'tigers', 'eagles', 'adversaries', 'timid', 'sheep', 'chickens', 'find', 'triumph', 'empty', 'victors', 'carried', 'enemy', 'slain', 'surrendered', 'cowering', 'utter', 'subjection', 'realize', 'toc', 'rival', 'friend', 'left', 'supreme', 'solitary', 'desolate', 'forlorn', 'find', 'triumph', 'tragedy', 'hero', 'always', 'exultant', 'may', 'proof', 'moral', 'supremacy', 'china', 'rest', 'world', 'look', 'ah', 'q', 'light', 'elated', 'fly', 'victory', 'without', 'strange', 'consequences', 'though', 'quite', 'time', 'seemed', 'flying', 'flew', 'tutelary', 'god', 'temple', 'would', 'normally', 'snored', 'soon', 'lay', 'evening', 'however', 'found', 'difficult', 'close', 'eyes', 'felt', 'something', 'matter', 'thumb', 'first', 'finger', 'seemed', 'smoother', 'usual', 'impossible', 'say', 'whether', 'something', 'soft', 'smooth', 'little', 'nun', 'face', 'stuck', 'fingers', 'whether', 'fingers', 'rubbe

In [9]:
#implementation phase
js_yg = jaccard_similarities(yiyun_list, google_list)
js_hg = jaccard_similarities(hsien_list,google_list)
js_hy = jaccard_similarities(hsien_list, yiyun_list)

print(js_yg, js_hg, js_hy)



18.58974358974359 21.49532710280374 26.126126126126124


In [10]:
#semantic analysis cosine similarity
hsien_list      = ','.join(str(v) for v in hsien_list)
google_list    = ','.join(str(v) for v in google_list)
yiyun_list    = ','.join(str(v) for v in yiyun_list)

In [14]:
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 6.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
#Calculating Cosine Similarity through TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

doc = [hsien_list, yiyun_list, google_list]

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(doc)
words = vectorizer.get_feature_names_out()
similarity_matrix = cosine_similarity(tfidf)

print(similarity_matrix)

[[1.         0.33609327 0.50282451]
 [0.33609327 1.         0.30499986]
 [0.50282451 0.30499986 1.        ]]
